---

## Ejercicio 1

Realiza Web Scraping de una pagina de la bolsa de Madrid usando BeautifulSoup y Selenium

---

Extraeremos de la bolsa de Madrid los datos basicos de las acciones de las compañias energeticas mas importantes. 

Lo haremos usando las 2 tecnicas indicadas: BeautifulSoup y Selenium.

In [1]:
from bs4 import BeautifulSoup
import requests

Definimos **isin_empr_ener** un diccionario con el codigo ISIN de las empresas energeticas mas importantes.

Completaremos la direccion de cotizacion de cada empresa con su codigo ISIN

**url_bolsa** tiene la direccion base de la cotizacion de las empresas

In [2]:
url_bolsa = "https://www.bolsamadrid.es/esp/aspx/Empresas/FichaValor.aspx?ISIN="

isin_empr_ener={'Iberdrola':'ES0144580Y14', 'Endesa':'ES0130670112', 'Naturgy':'ES0116870314', 'Repsol':'ES0173516115',
                'Acciona':'ES0105563003', 'Enagas':'ES0130960018', 'Red Electrica': 'ES0173093024',
                'Audax Renovables':'ES0136463017', 'SolarPack':'ES0105385001', 'Solaria': 'ES0165386014',
               'Green Energy': 'ES0105079000'}

Implementemos un bucle que extraiga de la pagina de cotizacion de cada empresa los datos de cotizacion de interes.

In [3]:
for k, v in isin_empr_ener.items():

    source = requests.get( url_bolsa + v).text
    soup = BeautifulSoup(source, 'lxml')

    company = soup.find('th', class_='Ult')
    web_site = company.a['href']
    capital_social = soup.find(id='ctl00_Contenido_CabEmisora_CapSocDat' ).text
    
    tabla_cotiz = soup.find(id='ctl00_Contenido_tblPrecios')
    cotiz_row = tabla_cotiz.find_all('tr')
    cotiz = cotiz_row[1].find_all('td')
    var = cotiz[4].text
    vol_ses = cotiz[9].text
    vol_efec = cotiz[10].text

    print('Nombre de la Compañia: ', company.text)
    print('Portal Internet:', web_site)
    print('Capital Social de', company.text, ': ', capital_social)
    print('Cotizacion:', cotiz[5].text)
    print('Variación: ', var, '%')
    print('Volumen Sesion:', vol_ses, '=',vol_efec,'€')
    print('Actualizado a:', cotiz[0].text ,'-',cotiz[1].text)
    print('-------------------------------------------------------------')

Nombre de la Compañia:  IBERDROLA, S.A.
Portal Internet: http://www.iberdrola.es
Capital Social de IBERDROLA, S.A. :  4.774.566.000,00 Euros
Cotizacion: 9,7500
Variación:  1,25 %
Volumen Sesion: 5.252.833 = 50.980.012,69 €
Actualizado a: 03/12/2021 - 11:15:16
-------------------------------------------------------------
Nombre de la Compañia:  ENDESA, SOCIEDAD ANONIMA
Portal Internet: http://www.endesa.com
Capital Social de ENDESA, SOCIEDAD ANONIMA :  1.270.502.540,40 Euros
Cotizacion: 19,4700
Variación:  0,10 %
Volumen Sesion: 164.305 = 3.196.125,15 €
Actualizado a: 03/12/2021 - 11:15:21
-------------------------------------------------------------
Nombre de la Compañia:  NATURGY ENERGY GROUP, S.A.
Portal Internet: https://www.naturgy.com
Capital Social de NATURGY ENERGY GROUP, S.A. :  969.613.801,00 Euros
Cotizacion: 24,5100
Variación:  0,25 %
Volumen Sesion: 99.043 = 2.420.592,61 €
Actualizado a: 03/12/2021 - 11:14:56
-------------------------------------------------------------
Nom

Ahora extraeremos la misma informacion bursatil de las compañias electricas pero utilizando Selenium.

Importamos las librerias necesarias.

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait


Usaremos como direccion base la url de la portada de la bolsa de Madrid

Indicamos la ubicacion del driver de control del navegador, que en este caso será Chrome

In [5]:
file_path = 'C:/Users/sergi/python/chromedriver.exe'

url='https://www.bolsamadrid.es/esp/aspx/Portada/Portada.aspx'

Hagamos un bucle para extraer la informacion bursatil de todas las empresas pero esta vez con Selenium.

Este bucle abrira un navegador para cada empresa, realizara la busqueda segun su ISIN y estraerá la información de interes.

In [6]:
for k, v in isin_empr_ener.items():
    
    s = Service(file_path)
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    
    search = driver.find_element(By.NAME, 'busq')
    search.send_keys(v)
    search.send_keys(Keys.RETURN)
    
    wait = WebDriverWait(driver, 2)
    
    table = driver.find_element(By.ID, "ctl00_Contenido_tblEmisoras")
    result = table.find_element(By.TAG_NAME, 'a')
    result.click()
    
    table2 = driver.find_element(By.ID, "ctl00_Contenido_tblPrecios")
    cells = table2.find_elements(By.TAG_NAME, 'td')
    
    table3 = driver.find_element(By.ID, "ctl00_Contenido_CabEmisora_tblEmisora")
    empresa = table3.find_element(By.CLASS_NAME, "Ult")
    
    link = table3.find_element(By.TAG_NAME, 'a')
    link_empr= link.get_attribute('href')
    
    cap_soc = driver.find_element(By.ID,"ctl00_Contenido_CabEmisora_CapSocDat" )
    
    print('Nombre de la Compañia: ', empresa.text)
    print('Portal Internet:', link_empr)
    print('Capital Social de', empresa.text, ': ', cap_soc.text)
    print('Cotizacion:', cells[5].text)
    print('Variación: ', cells[4].text, '%')
    print('Volumen Sesion:', cells[9].text, '=',cells[10].text,'€')
    print('Actualizado a:', cells[0].text ,'-',cells[1].text)
    print('-------------------------------------------------------------')
    
    # cerramos la pestaña del navegador
    driver.close()
    

Nombre de la Compañia:  IBERDROLA, S.A.
Portal Internet: http://www.iberdrola.es/
Capital Social de IBERDROLA, S.A. :  4.774.566.000,00 Euros
Cotizacion: 9,7480
Variación:  1,23 %
Volumen Sesion: 5.260.003 = 51.049.905,28 €
Actualizado a: 03/12/2021 - 11:16:31
-------------------------------------------------------------
Nombre de la Compañia:  ENDESA, SOCIEDAD ANONIMA
Portal Internet: http://www.endesa.com/
Capital Social de ENDESA, SOCIEDAD ANONIMA :  1.270.502.540,40 Euros
Cotizacion: 19,4750
Variación:  0,13 %
Volumen Sesion: 168.033 = 3.268.729,20 €
Actualizado a: 03/12/2021 - 11:16:11
-------------------------------------------------------------
Nombre de la Compañia:  NATURGY ENERGY GROUP, S.A.
Portal Internet: https://www.naturgy.com/
Capital Social de NATURGY ENERGY GROUP, S.A. :  969.613.801,00 Euros
Cotizacion: 24,5100
Variación:  0,25 %
Volumen Sesion: 99.043 = 2.420.592,61 €
Actualizado a: 03/12/2021 - 11:14:56
-------------------------------------------------------------


--------------------

## Ejercicio 2

Documenta en un word la informacion generada con el web scrapping.

-------------------

Guardamos en un csv la informacion bursatil de las empresas energeticas

Modificamos el bucle para guardar la informacion en el archivo csv en vez de dar la salida por el terminal

In [7]:
import csv

In [8]:
csv_file = open('datos_bolsa.csv', 'w', newline='')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Empresa', 'Página Web', 'Capital Social', 'Cotización',
                     'Variación', 'Volumen acciones', 'Volumen Euros', 'Fecha',
                    'Hora'])

98

In [9]:
for k, v in isin_empr_ener.items():

    source = requests.get( url_bolsa + v).text
    soup = BeautifulSoup(source, 'lxml')

    company = soup.find('th', class_='Ult') 
    web_site = company.a['href']
    capital_social = soup.find(id='ctl00_Contenido_CabEmisora_CapSocDat' ).text

    tabla_cotiz = soup.find(id='ctl00_Contenido_tblPrecios')
    cotiz_row = tabla_cotiz.find_all('tr')
    cotiz = cotiz_row[1].find_all('td')

    var = cotiz[4].text
    vol_ses = cotiz[9].text
    vol_efec = cotiz[10].text
    
    csv_writer.writerow([company.text, web_site, capital_social, cotiz[5].text, var+'%',
                        vol_ses, vol_efec+'€', cotiz[0].text, cotiz[1].text])
    
csv_file.close()

El archivo csv lo podemos abrir en excel y posteriormente lo podemos adjuntar en un archivo word. Se adjuntan los archivos csv, excel y word.

---

## Ejercicio 3

Elige una web y haz web scraping con Scrapy.

---

Para este ejemplo haremos scraping de la web de la bolsa de Madrid de Iberdrola.

Creamos el proyecto de Scrapy llamado **bolsa**

Creamos el **spider bolsaspider** para obtener la misma informacion bursatil que obteniamos con BeautifulSoup o Selenium.


In [10]:
import scrapy

In [ ]:
class BolsaSpider(scrapy.Spider):
    name = 'bolsaspider'
    allowed_domains = ['www.bolsamadrid.es']
    start_urls = ['https://www.bolsamadrid.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0144580Y14']
    
    def parse(self, response):
        nom = response.css('th.Ult')
        comp = nom.css('a::text').get()
        
        web_link= nom.css('a::attr(href)').get()
        cap = response.css('[id="ctl00_Contenido_CabEmisora_CapSocDat"]::text').get()
        
        tabla = response.css('[id="ctl00_Contenido_tblPrecios"]')
        cot = tabla.css('td::text')[5].get()
        var = tabla.css('td::text')[4].get()
        vol1 = tabla.css('td::text')[9].get()
        vol2 = tabla.css('td::text')[10].get()
        day = tabla.css('td::text')[0].get()
        h = tabla.css('td::text')[1].get()
        
        print('Nombre de la Compañia: ', comp)
        print('Portal Internet:', web_link)
        print('Capital Social de', comp, ': ', cap)
        print('Cotizacion:', cot)
        print('Variación: ', var, '%')
        print('Volumen Sesion:', vol1, '=',vol2,'€')
        print('Actualizado a:', day ,'-', h)
        print('-------------------------------------------------------------')
   

Tras ejecutar en el terminal de windows el spider bolsaspider, obtenemos la siguiente salida por el mismo terminal:

In [11]:
print('''bolsa> scrapy crawl bolsaspider
2021-12-02 18:13:40 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: bolsa)''')

bolsa> scrapy crawl bolsaspider
2021-12-02 18:13:40 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: bolsa)


In [12]:
print('''Nombre de la Compañia:  IBERDROLA, S.A.
Portal Internet: http://www.iberdrola.es
Capital Social de IBERDROLA, S.A. :  4.774.566.000,00 Euros
Cotizacion: 9,6300
Variación:  -4,27 %
Volumen Sesion: 19.568.647 = 190.236.280,37 €
Actualizado a: 02/12/2021 - 17:35:19''')

Nombre de la Compañia:  IBERDROLA, S.A.
Portal Internet: http://www.iberdrola.es
Capital Social de IBERDROLA, S.A. :  4.774.566.000,00 Euros
Cotizacion: 9,6300
Variación:  -4,27 %
Volumen Sesion: 19.568.647 = 190.236.280,37 €
Actualizado a: 02/12/2021 - 17:35:19


Se adjunta otros logs proporcionados en la ejecucion del spider

In [13]:
print('''2021-12-02 18:13:40 [scrapy.core.engine] INFO: Closing spider (finished)
2021-12-02 18:13:40 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 497,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 60332,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 1,
 'downloader/response_status_count/404': 1,
 'elapsed_time_seconds': 0.314713,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 12, 2, 17, 13, 40, 931096),
 'httpcompression/response_bytes': 73599,
 'httpcompression/response_count': 1,
 'log_count/DEBUG': 72,
 'log_count/INFO': 10,
 'response_received_count': 2,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/404': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2021, 12, 2, 17, 13, 40, 616383)}
2021-12-02 18:13:40 [scrapy.core.engine] INFO: Spider closed (finished)''')

2021-12-02 18:13:40 [scrapy.core.engine] INFO: Closing spider (finished)
2021-12-02 18:13:40 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 497,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 60332,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 1,
 'downloader/response_status_count/404': 1,
 'elapsed_time_seconds': 0.314713,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 12, 2, 17, 13, 40, 931096),
 'httpcompression/response_bytes': 73599,
 'httpcompression/response_count': 1,
 'log_count/DEBUG': 72,
 'log_count/INFO': 10,
 'response_received_count': 2,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/404': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2021, 12, 2, 17, 13, 40, 616383)}
2021-